In [9]:
import sys
import math

frequenciesList = []
finalDictionary = {}
def file_character_frequencies(file_name):
    word_list = []

    file = open(file_name, 'rU')

    for line in file:
        for word in line:
           word_list.append(word);

    letter_frequency = {}

    for word in word_list:
        for letter in word:
            keys = letter_frequency.keys()
            if letter in keys:
                letter_frequency[letter] += 1
            else:
                letter_frequency[letter] = 1

    return letter_frequency


class PriorityTuple(tuple):
    """A specialization of tuple that compares only its first item when sorting.
    Create one using double parens e.g. PriorityTuple((x, (y, z))) """
    def __lt__(self, other):
        return self[0] < other[0]

    def __le__(self, other):
        return self[0] <= other[0]

    def __gt__(self, other):
        return self[0] > other[0]

    def __ge__(self, other):
        return self[0] >= other[0]

    def __eq__(self, other):
        return self[0] == other[0]

    def __ne__(self, other):
        x = self.__eq__(other)
        return not x

def huffman_codes_from_frequencies(frequencies):
    if (len(frequencies) == 2):
        return dict(zip(frequencies.keys(), ['0', '1']))

    for key, value in frequencies.items():
        frequenciesList.append((value, key))

    #print("Freq before buildHeap {}".format(frequenciesList))
    buildHeap(frequenciesList)
    #print("Freq after buildHeap {}".format(frequenciesList))

    for currentValue in range(len(frequenciesList) - 1):
        t1 = heapExtractMin(frequenciesList)
        t2 = heapExtractMin(frequenciesList)
        # best way to merge?
        mergedtuple = PriorityTuple((t1[0]+t2[0], t1, t2))
        heapInsert(frequenciesList, mergedtuple)
        s = ""
        dfs(frequenciesList[0], s)

        return finalDictionary


def huffman_letter_codes_from_file_contents(file_name):
    """WE WILL GRADE BASED ON THIS FUNCTION."""
    # Suggested strategy...
    freqs = file_character_frequencies(file_name)
    return huffman_codes_from_frequencies(freqs)
    #return {}


def encode_file_using_codes(file_name, letter_codes):
    """Provided to help you play with your code."""
    contents = ""
    with open(file_name) as f:
        contents = f.read()
    file_name_encoded = file_name + "_encoded"
    with open(file_name_encoded, 'w') as fout:
        for c in contents:
            fout.write(letter_codes[c])
    print("Wrote encoded text to {}".format(file_name_encoded))


def decode_file_using_codes(file_name_encoded, letter_codes):
    """Provided to help you play with your code."""
    contents = ""
    with open(file_name_encoded) as f:
        contents = f.read()
    file_name_encoded_decoded = file_name_encoded + "_decoded"
    codes_to_letters = {v: k for k, v in letter_codes.items()}
    with open(file_name_encoded_decoded, 'w') as fout:
        num_decoded_chars = 0
        partial_code = ""
        while num_decoded_chars < len(contents):
            partial_code += contents[num_decoded_chars]
            num_decoded_chars += 1
            letter = codes_to_letters.get(partial_code)
            if letter:
                fout.write(letter)
                partial_code = ""
    print("Wrote decoded text to {}".format(file_name_encoded_decoded))


# BEGIN DO NOT MODIFY
#########HEAP#####################################

db = False
swap_count = 0
heapify_call_count = 0

array_length = 0 # ONCE SET FOR A GIVEN ARRAY, THIS SHOULD NEVER CHANGE
# EVEN WHEN DOING OPERATIONS LIKE EXTRACT-MIN AND HEAP-INSERT

n = 0 #length of the 'heap'. n <= array_length

def reset_counts():
    global swap_count
    swap_count = 0
    global heapify_call_count
    heapify_call_count = 0


def swap(A, i, j):
    global swap_count
    swap_count += 1
    A[i], A[j] = A[j], A[i]


def count_heapify():
    global heapify_call_count
    heapify_call_count += 1


def current_counts():
    return {'swap_count': swap_count, 'heapify_call_count': heapify_call_count}

def readNums(filename):
    """Reads a text file containing whitespace separated numbers.
    Returns a list of those numbers."""
    global array_length
    global n

    n = 0 # set initial heap length to 0
    with open(filename) as f:
        lst = [int(x) for line in f for x in line.strip().split() if x]
        array_length = len(lst)

        if db:
            print("List read from file {}: {}".format(filename, lst))
            print("array_length = {}".format(array_length))

        return lst

def shuffled_list(length, seed):
    global array_length
    global n

    lst = list(range(10, length + 10))
    import random
    r = random.Random(seed) # pseudo random, so it is repeatable
    r.shuffle(lst)
    array_length = len(lst)
    n = 0 # set initial heap length to 0
    return lst

def printCompleteTree(A):
    """ A handy function provided to you, so you can see a
    complete tree in its proper shape.
    Note that this function shows the ENTIRE ARRAY in the form of a tree
    """

    height = int(math.log(len(A), 2))
    width = len(str(max(A)))
    for i in range(height + 1):
        print(width * (2 ** (height - i) - 1) * " ", end="")
        for j in range(2 ** i):
            idx = 2 ** i - 1 + j
            if idx >= len(A):
                print()
                break
            if j == 2 ** i - 1:
                print("{:^{width}}".format(A[idx], width=width))
            else:
                print("{:^{width}}".format(A[idx], width=width),
                      width * (2 ** (height - i + 1) - 1) * " ", sep='', end='')
    print()


def report_counts_on_basic_ops(A, loop_extracts=1, loop_inserts=1):
    original_len = len(A)
    print("\nREPORT on list of len: {}".format(original_len))
    reset_counts()
    buildHeap(A)
    print("buildHeap(A):           \t", current_counts())

    reset_counts()
    m = heapExtractMin(A)
    print("heapExtractMin(A) => {}:\t".format(m), current_counts())

    reset_counts()
    heapInsert(A, m)
    print("heapInsert(A, {}):       \t".format(m), current_counts())

    for i in range(loop_extracts):
        reset_counts()
        m = heapExtractMin(A)
        print("heapExtractMin(A) => {}:\t".format(m), current_counts())

    import random
    r = random.Random(0)
    for i in range(loop_inserts):
        reset_counts()
        new_number = r.randrange(0, original_len // 8)
        heapInsert(A, new_number)
        print("heapInsert(A, {}):       \t".format(new_number), current_counts())

# heaps here are complete binary trees allocated in arrays (0 based)
def parent(i):
    return (i - 1) // 2


def left(i):
    return 2 * i + 1


def right(i):
    return left(i) + 1

def heapify(A, i):
    """Ensure that the tree rooted at element i in the list A is a heap,
    assuming that the trees rooted at elements left(i) and right(i) are already
    heaps. Obviously, if left(i) or right(i) are >= n, then element i simply does
    not have those out-of-bounds children. In order to implement an in-place heap sort,
    we will sometimes need to consider the tail part of A as out-of-bounds, even though
    elements do exist there. So instead of comparing with len(A), use the parameter n to
    determine if the child "exists" or not.

    Since the (up to) two child trees are already heaps, we just need to find the right
    place for the element at i. If it is smaller than both its children, then nothing
    more needs to be done, it's already a min heap. Otherwise you should swap the root
    with the smallest child and recursively heapify that tree.

    If you determine that the element at i should swap with one of its children nodes,
    MAKE SURE you do this by calling the swap function defined above.
    """

    count_heapify() # This should be the first line of the heapify function, do not change.
    l = left(i)
    r = right(i)
    smallest = i
    if l < n and A[l] < A[smallest]:
        smallest = l
    if r < n and A[r] < A[smallest]:
        smallest = r
    
    if smallest != i:
        swap(A,i,smallest)
        heapify(A, smallest)
    # Your code here
    
    


def buildHeap(A):
    """Turn the list A (whose elements could be in any order) into a
    heap. Call heapify on all the internal nodes, starting with
    the last internal node, and working backwards to the root.
    """

    
    global n
    n = len(A) # The entire list A will be turned into a heap.
    # So length of heap = length of array
    i = parent(n-1)
    while i>= 0 :
        heapify(A,i)
        i=i-1
    # Your code here


def heapExtractMin(A):
    """Extract the min element from the heap A. Make sure that A
    is a valid heap afterwards. Return the extracted element.
    MAKE SURE you are not extracting from an empty heap.
    This operation should perform approximately log_2(n)
    comparisons and swaps (heapify calls and swap calls).
    Your implementation should not perform O(n) (linear) work.
    Length of the array, A, should not change.
    MAKE SURE you swap elements by calling the swap function defined above.
    """
    global n
    global swap_count 
    
    if(n<=0):
        return
    if(n==1):
        return
    min = A[0]
    A[0]= None
    swap_count= swap_count-1
    swap(A, 0, n-1)
    n=n-1
    heapify(A, 0)

    return min

    
    # Your code here
    


def heapInsert(A, v):
    """Insert the element v into the heap A. Make sure that A
    is a valid heap afterwards.
    This operation should perform approximately log_2(n)
    comparisons and swaps (swap calls).
    Your implementation should not perform O(n) (linear) work.
    MAKE SURE you swap elements by calling the swap function defined above.
    Always ensure n <= array_length (cannot insert when heap is full)
    """
          
    # Your code here
    global n
    if(n>= len(A)):
    	return
    A[n] = v
    start = n
    n = n+ 1
    while parent(start) > -1 and A[parent(start)] > v:
        #A[start], A[parent(start)] = A[parent(start)], A[start]
        swap(A, start, parent(start))
        start = parent(start)



    



def main():
    import pprint
    frequencies = file_character_frequencies(sys.argv[1])
    #pprint.pprint(frequencies)
    codes = huffman_codes_from_frequencies(frequencies)
    #pprint.pprint(codes)
    encode_file_using_codes(sys.argv[1],codes)
    decode_file_using_codes(sys.argv[1]+"_encoded",codes)
    


if __name__ == '__main__':
    """We are NOT grading you based on main, this is for you to play with."""
    main()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: 'U' mode is deprecated
  if __name__ == '__main__':


FileNotFoundError: [Errno 2] No such file or directory: '-f'